In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
from utils import * 
import load_data
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
import os
from training_audio_model import *

# Set the device for GPU usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

# Load the data
transcr_path = 'paco-cheese/transcr'
data = load_data.load_all_ipus(folder_path=transcr_path, load_words=True)

# Display the first 3 entries of the data
data[:3]

# Import additional utilities and models
from utils import *
from training_text_model import *

In [ ]:

df = pd.DataFrame(data)

# Create the target variable 'y' from the DataFrame
y = create_y(df)

y de base
9131
7269
y yield at
3090
13310


'\nprint("y turn after")\ny=create_y_turn_after(df) # Création de la target\nprint(y.count(1))\nprint(y.count(0))\nprint(len(y))\nprint(len(data))\n'

# Importation des données audios

In [ ]:
# Define audio files path
audio_files_path = 'paco-cheese/audio/2_channels/'

# Extract audio segments
audio_segments = extract_audio_segments(data,audio_files_path)

# Extract features from audio segments
X = np.array([extract_features(segment) for segment in audio_segments])

In [ ]:
# Create an AudioDataset from the features and labels
dataset = AudioDataset(X, y)

#train_size = int(0.8 * len(dataset))
#test_size = len(dataset) - train_size
#train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Determine the size of the training set (80% of the total data)
train_size = int(0.8 * len(dataset))

# Split the features and labels into training and testing sets
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create AudioDatasets for the training and testing sets
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

# Create DataLoaders for the training and testing sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader_audio = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Display
print(len(y))
print(len(y_test))

In [ ]:
# Load the trained model
model = torch.load('modele/model_audio')

# Use the model to make predictions on the test set
all_preds_audio, all_labels = predition_model_audio(model, test_loader_audio, device, proba=False)

# Calculate the weighted F1 score
f1 = f1_score(all_labels, all_preds_audio, average='weighted')

# Generate the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds_audio)

# Print the F1 score and confusion matrix
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculate the total number of instances for each class
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])

# Calculate the number of correctly detected instances for each class
detected_class_0 = conf_matrix[0, 0]  # True positives for class 0
detected_class_1 = conf_matrix[1, 1]  # True positives for class 1

# Print the number of correctly detected instances for each class
print(f'Number of class 0 instances detected: {detected_class_0} out of {total_class_0}')
print(f'Number of class 1 instances detected: {detected_class_1} out of {total_class_1}')

In [ ]:
# Use the model to make predictions on the test set, return probabilities
all_preds_audio, all_labels_audio = predition_model_audio(model, test_loader_audio, device, proba=True)

# Print the predicted probabilities and their count
print(all_preds_audio)
print(len(all_preds_audio))

# Importation données texte

In [ ]:
# Convert the 'text_words' column of the data to a list
features = data['text_words'].to_numpy().reshape(-1).tolist()  

# Replace any null values in the features with '[UNK]'
features = [word if not pd.isnull(word) else '[UNK]' for word in features]

# Determine the size of the training set (80% of the total data)
train_size = int(len(features) * 0.8)

# Split the features and labels into training and testing sets
X_train = features[:train_size]
y_train = y[:train_size]
X_test = features[train_size:]
y_test = y[train_size:]

# Define the window size for creating sequences
window_size = 5

# Create sequences from the training and testing features
X_train = create_sequences(X_train)
X_test = create_sequences(X_test)

# Define the parameters for the model
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the tokenizer for CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)

# Create TextDatasets and DataLoaders for the training and testing sets
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader_text = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader_text = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## prediction modèle texte

In [ ]:
# Load the trained model
model = torch.load('modele/camembert_epoch_3.bin')

# Set the model to evaluation mode
model.eval()

# Use the model to make predictions on the test set
all_preds_text, all_labels = prediction_model_text(model, test_loader_text, device, proba=False)

# Calculate the weighted F1 score
f1 = f1_score(all_labels, all_preds_text, average='weighted')

# Generate the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds_text)

# Print the F1 score and confusion matrix
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculate the total number of instances for each class
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])

# Calculate the number of correctly detected instances for each class
detected_class_0 = conf_matrix[0, 0]  # True positives for class 0
detected_class_1 = conf_matrix[1, 1]  # True positives for class 1

# Print the number of correctly detected instances for each class
print(f'Number of class 0 instances detected: {detected_class_0} out of {total_class_0}')
print(f'Number of class 1 instances detected: {detected_class_1} out of {total_class_1}')

# prédiction late fusion

In [ ]:
# Load the trained text model
model_text = torch.load('modele/camembert_epoch_3.bin')

# Set the model to evaluation mode
model_text.eval()

# Use the model to make predictions on the test set, return probabilities
all_preds_text_proba, all_labels = prediction_model_text(model_text, test_loader_text, device, proba=True)

# Combine the predictions from the text and audio models using a logical AND operation
# combined_preds_and = [pred_text and pred_audio for pred_text, pred_audio in zip(all_preds_text, all_preds_audio)]

In [ ]:
# Load the trained audio model
model_audio = torch.load('modele/model_audio')

# Use the model to make predictions on the test set, return probabilities
all_preds_audio_proba, all_labels_audio = predition_model_audio(model_audio, test_loader_audio, device, proba=True)

In [ ]:
# Display
print(all_preds_audio)
print(all_preds_text)

In [ ]:
# Add the probability arrays from the audio and text models
sum_array = np.add(all_preds_audio_proba, all_preds_text_proba)

# Determine the class (0 or 1) for each list by finding the index of the maximum value
combined_preds = np.argmax(sum_array, axis=1)

In [ ]:
# Calculate the weighted F1 score for the combined predictions
f1 = f1_score(all_labels, combined_preds, average='weighted')

# Generate the confusion matrix for the combined predictions
conf_matrix = confusion_matrix(all_labels, combined_preds)

# Print the F1 score and confusion matrix
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculate the total number of instances for each class
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])

# Calculate the number of correctly detected instances for each class
detected_class_0 = conf_matrix[0, 0]  # True positives for class 0
detected_class_1 = conf_matrix[1, 1]  # True positives for class 1

# Print the number of correctly detected instances for each class
print(f'Number of class 0 instances detected: {detected_class_0} out of {total_class_0}')
print(f'Number of class 1 instances detected: {detected_class_1} out of {total_class_1}')